In [1]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
import dagshub

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

import os

import warnings
warnings.filterwarnings('ignore')


In [2]:
mlflow.set_tracking_uri("https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow")
dagshub.init(repo_owner='sumitwatkar', repo_name='Text-Classification-using-MLOPS', mlflow=True)

Accessing as sumitwatkar

Initialized MLflow to track repo "sumitwatkar/Text-Classification-using-MLOPS"

Repository sumitwatkar/Text-Classification-using-MLOPS initialized!

In [3]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
# Normalize the text data
df = normalize_text(df)

In [6]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Set the experiment name
mlflow.set_experiment("LoR Hyperparameter Tuning")

# Define hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

2024/08/10 00:03:41 INFO mlflow.tracking.fluent: Experiment with name 'LoR Hyperparameter Tuning' does not exist. Creating a new experiment.


In [11]:
# Start the parent run for hyperparameter tuning
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook
    notebook_path = "exp3_lor_bow_hp.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

Mean CV Score: 0.7051163373017877, Std CV Score: 0.014297013862890536
Accuracy: 0.739277108433735
Precision: 0.7749419953596288
Recall: 0.6581280788177339
F1 Score: 0.7117741076185402


2024/08/10 00:04:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3/runs/ecb34e22071d4a22bd61b1004ad9bc6e.
2024/08/10 00:04:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3.


Mean CV Score: 0.7811972314475085, Std CV Score: 0.011496238130202462
Accuracy: 0.7893975903614457
Precision: 0.7816764132553606
Recall: 0.7901477832512315
F1 Score: 0.7858892699657031


2024/08/10 00:04:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3/runs/651abfa87e5743e68d38b7cf0daeb6ac.
2024/08/10 00:04:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3.


Mean CV Score: 0.7840436153359088, Std CV Score: 0.009188273790904653
Accuracy: 0.7821686746987951
Precision: 0.7735665694849369
Recall: 0.7842364532019704
F1 Score: 0.7788649706457925


...

2024/08/10 00:05:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3/runs/0373939806334f30a9b3d7df2615683d.
2024/08/10 00:05:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3.


Mean CV Score: 0.781002998936019, Std CV Score: 0.005279351561572635
Accuracy: 0.7836144578313253
Precision: 0.7715930902111324
Recall: 0.7921182266009852
F1 Score: 0.7817209528439475


2024/08/10 00:05:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3/runs/87c98ef9934243eaa0056e3fcffccd2b.
2024/08/10 00:05:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3.


Best Params: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Best F1 Score: 0.7901458706737883


2024/08/10 00:05:37 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/10 00:05:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run upbeat-bear-420 at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3/runs/6534465c92cc45578c7da85dbdc52a5a.
2024/08/10 00:05:37 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sumitwatkar/Text-Classification-using-MLOPS.mlflow/#/experiments/3.
